In [1]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d30f10c37885dc6199e8e528da2a68f1cd2adf80929eddd0116fedf44c084cae
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import random
import sys
import os
from nltk.translate.bleu_score import sentence_bleu
import json
import codecs
json_load = lambda x: json.load(codecs.open(x, 'r', encoding='utf-8'))
json_dump = lambda d, p: json.dump(d, codecs.open(p, 'w', 'utf-8'), indent=2, ensure_ascii=False)
import numpy as np
from nltk import word_tokenize
import nltk
nltk.download('punkt')
from rouge_score import rouge_scorer
from scipy import interpolate
from pathlib import Path
from typing import List, Union, Iterable
from itertools import zip_longest
from collections import defaultdict
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
rouge_map = {
    "rouge1": [{"length": 3.984911270091982, "score": 0.06441979516467827},
{"length": 5.958849763077208, "score": 0.09805036171757445},
{"length": 7.908073957075165, "score": 0.1264078574240267},
{"length": 9.815376753693208, "score": 0.14799855615831176},
{"length": 11.669200037164362, "score": 0.16462272749307244},
{"length": 13.424593514819287, "score": 0.1780456954008045},
{"length": 15.064944718015425, "score": 0.18711340658961978},
{"length": 16.568800520301032, "score": 0.19360401553241696},
{"length": 17.935250394871318, "score": 0.1981390956396358},
{"length": 19.17587103967295, "score": 0.2028477580052331},
{"length": 20.205899842051473, "score": 0.20568411912661153},
{"length": 21.183452568986343, "score": 0.20744671526261746},
{"length": 21.977924370528665, "score": 0.20841666128479538},
{"length": 26.75476168354548, "score": 0.21086241474611672},
{"length": 52.85634116881911, "score": 0.2331159448484293},
{"length": 77.34764470872433, "score": 0.22772254093882188},
{"length": 99.4468085106383, "score": 0.21670611535128495},
{"length": 118.99010498931523, "score": 0.20656306793005602},
{"length": 135.94575861748584, "score": 0.1984243073734219},
{"length": 150.71153953358728, "score": 0.19204958051321147},
{"length": 163.3066710025086, "score": 0.1871192863560473},
{"length": 174.03108798662086, "score": 0.1834498432953618},
{"length": 183.1557837034284, "score": 0.1806604090047676}],
    "rouge2":[{"length": 3.984911270091982, "score": 0.012204318074850343},
{"length": 5.958849763077208, "score": 0.023002475506111737},
{"length": 7.908073957075165, "score": 0.03455646883959736},
{"length": 9.815376753693208, "score": 0.043565859835247536},
{"length": 11.669200037164362, "score": 0.05076404631862093},
{"length": 13.424593514819287, "score": 0.056914830457828657},
{"length": 15.064944718015425, "score": 0.06113690491038822},
{"length": 16.568800520301032, "score": 0.06379210133931756},
{"length": 17.935250394871318, "score": 0.06558818891078552},
{"length": 19.17587103967295, "score": 0.06856054937431301},
{"length": 20.205899842051473, "score": 0.07002250650151698},
{"length": 21.183452568986343, "score": 0.0707186916351147},
{"length": 21.977924370528665, "score": 0.07123874205675668},
{"length": 26.75476168354548, "score": 0.0737207868745392},
{"length": 52.85634116881911, "score": 0.08887700947793167},
{"length": 77.34764470872433, "score": 0.09367539519260709},
{"length": 99.4468085106383, "score": 0.09468673311905121},
{"length": 118.99010498931523, "score": 0.09422323872038459},
{"length": 135.94575861748584, "score": 0.09365316220876337},
{"length": 150.71153953358728, "score": 0.09293310736739671},
{"length": 163.3066710025086, "score": 0.09214270076293635},
{"length": 174.03108798662086, "score": 0.09153337352167813},
{"length": 183.1557837034284, "score": 0.09106333800837592}],
"rougeL":[{"length": 3.984911270091982, "score": 0.0603069994104717},
{"length": 5.958849763077208, "score": 0.08865593606672337},
{"length": 7.908073957075165, "score": 0.11183012948421223},
{"length": 9.815376753693208, "score": 0.12830974766110848},
{"length": 11.669200037164362, "score": 0.14071397628227003},
{"length": 13.424593514819287, "score": 0.14983513211146812},
{"length": 15.064944718015425, "score": 0.15586398800876933},
{"length": 16.568800520301032, "score": 0.15967759189458222},
{"length": 17.935250394871318, "score": 0.16221029161539405},
{"length": 19.17587103967295, "score": 0.16514231726929202},
{"length": 20.205899842051473, "score": 0.16692078881009273},
{"length": 21.183452568986343, "score": 0.16762166739283607},
{"length": 21.977924370528665, "score": 0.16775785324386341},
{"length": 26.75476168354548, "score": 0.16851432006298805},
{"length": 52.85634116881911, "score": 0.17676076320959805},
{"length": 77.34764470872433, "score": 0.17027247886644845},
{"length": 99.4468085106383, "score": 0.16223875947025201},
{"length": 118.99010498931523, "score": 0.15543184730581103},
{"length": 135.94575861748584, "score": 0.15041118174002338},
{"length": 150.71153953358728, "score": 0.14662785591147373},
{"length": 163.3066710025086, "score": 0.14359477578260552},
{"length": 174.03108798662086, "score": 0.1415072129787698},
{"length": 183.1557837034284, "score": 0.13990352023938848}]
}

In [41]:
class ScicapSummarizationEval():
    def __init__(self):
        self.metric_list = ["rouge1", "rouge2", "rougeL"]
        self.rouge = rouge_scorer.RougeScorer(
            self.metric_list,
            use_stemmer=True,
        )

        # build score normalization
        
        self.normalization_funcs = {}
        for metric in self.metric_list:
            random_data = rouge_map[metric]
            length_list = np.array([d["length"] for d in random_data])
            score_list = np.array([d["score"] for d in random_data])
            normalization_func = interpolate.interp1d(
                length_list,
                score_list,
                bounds_error=False,
                fill_value="extrapolate",
            )
            self.normalization_funcs[metric] = normalization_func
        
    def clean_text(self, text):
        return text.lower()

    def evaluate(
        self, 
        hypothesis: List[str],
        reference: List[str],
    ):
        assert len(hypothesis) == len(reference), "The number of hypothesis and reference must be the same."

        score_list = []
        for hyp, ref in zip(hypothesis, reference):
            hyp = self.clean_text(hyp)
            ref = self.clean_text(ref)

            # compute scores and match
            scores = self.rouge.score(ref, hyp)
            bleu = sentence_bleu([ref.split()], hyp.split())
            score_dict = {
                "rouge1": scores["rouge1"].fmeasure,
                "rouge2": scores["rouge2"].fmeasure,
                "rougeL": scores["rougeL"].fmeasure,
                "length": len(word_tokenize(hyp)),
                "bleu-4": bleu
            }
            score_list.append(score_dict)
        
        # compute mean and normalize scores
        score_dict = {
            "length": np.mean([d["length"] for d in score_list]),
        }
        for metric in self.metric_list:
            score_dict[metric] = np.mean([d[metric] for d in score_list])
            score_dict[f"{metric}_normalized"] = score_dict[metric] / self.normalization_funcs[metric](score_dict["length"])
        score_dict["bleu-4"] = np.mean([d["bleu-4"] for d in score_list])
        return score_dict
        
def evaluate(test_annotation_file, user_submission_file, **kwargs):
    print("Starting Evaluation.....")
    #for metric, score in cocoEval.eval.items():
    #    print(metric, score)

    # load data
    ground_truth = json_load(test_annotation_file)
    submission = json_load(user_submission_file)
    
    print("align annotations and submission file..")
    # align data
    data = {sample["image_id"]: sample for sample in ground_truth["annotations"]}
    for prediction in submission:
        image_id = prediction["image_id"]
        data[image_id]["prediction"] = prediction["caption"]

    # convert to list
    hypothesis = []
    reference = []
    print(data)
    for image_id, sample in data.items():
        try:
            hypothesis.append(sample["prediction"])
            reference.append(sample["caption"])
        except:
          print("Missing id: ", image_id)
          return

    # evaluate
    evaluator = ScicapSummarizationEval()
    score = evaluator.evaluate(
        hypothesis=hypothesis,
        reference=reference,
    )

    output = {}
    output["result"] = [
        {
            "test_split": {
                "BLEU-4": score['bleu-4'],
                "Rouge-1": score['rouge1'],
                "Rouge-1-normalized": score['rouge1_normalized'],
                "Rouge-2": score['rouge2'],
                "Rouge-2-normalized": score['rouge2_normalized'],
                "Rouge-L": score['rougeL'],
                "Rouge-L-normalized": score['rougeL_normalized'],
            }
        }
    ]
    # To display the results in the result file
    output["submission_result"] = output["result"][0]
    print("Completed evaluation!")
    return output

In [47]:
evaluate("/content/test_annotations_devsplit.json", "/content/submission_dev.json")

Starting Evaluation.....
align annotations and submission file..
{7424363: {'image_id': 7424363, 'id': 8492163, 'caption': 'Fig. 1. The coefficient of Pe2 in the small-Pe optimal enhancement (32) with n = 1 (first zero). The optimal enhancement is achieved for m = 2, then drops off slowly.', 'caption_no_index': 'The coefficient of Pe2 in the small-Pe optimal enhancement (32) with n =  (first zero). The optimal enhancement is achieved for m = 2, then drops off slowly.', 'paragraph': ['Since j m,n increases monotonically with n, we must take n = 1 to minimize T . Thus the optimal streamlines pattern displays a single cell in the radial direction. It is then a simple matter of enumerating the zeros j m,1 to find that the integrated mean exit time is minimized for (m, n) = (2, 1), independent of the Péclet number (see Figure 1). The streamline and mean exit time patterns are illustrated in Figure 2.'], 'mention': [['It is then a simple matter of enumerating the zeros j m,1 to find that the

{'result': [{'test_split': {'BLEU-4': 0.25042009669367926,
    'Rouge-1': 0.5714285714285715,
    'Rouge-1-normalized': 3.0597864207058603,
    'Rouge-2': 0.5499999999999999,
    'Rouge-2-normalized': 9.020867687902546,
    'Rouge-L': 0.5714285714285715,
    'Rouge-L-normalized': 3.6718232461202764}}],
 'submission_result': {'test_split': {'BLEU-4': 0.25042009669367926,
   'Rouge-1': 0.5714285714285715,
   'Rouge-1-normalized': 3.0597864207058603,
   'Rouge-2': 0.5499999999999999,
   'Rouge-2-normalized': 9.020867687902546,
   'Rouge-L': 0.5714285714285715,
   'Rouge-L-normalized': 3.6718232461202764}}}